In [1]:
import numpy as np
import pandas as pd
import sys
import re
from functions import *
from collections import Counter
np.set_printoptions(precision=2)
pd.set_option('precision', 2)
%matplotlib inline
matplotlib.style.use('ggplot')

### NOTICE: The functions demoed in this notebook are now stored in mining_functions.py
#### (they may be outdated here)

In [2]:
PATH = '/Google Drive/Jonathan Sarah Ido folder/data/CCK/'
def get_path(path = PATH):
    if os.name == 'posix':
        return os.environ['HOME']+path #'/Google Drive/Jonathan Sarah Ido folder/data/CCK/'
    elif os.name == 'nt':
        return os.environ['USERPROFILE']+ path.replace('/','\\') #'\\Google Drive\Jonathan Sarah Ido folder\data\CCK\\'
    else:
        raise Exception('OS not recongnized. I\'m confused.')
gitpath = '/Documents/git/Phet-log-analyzer/cck/raw_data_parsing_check/'
df = pd.read_csv(get_path(path = gitpath) + 'phet_cck_user_actions+sophistication_WITHPAUSE_more_circuit_info.txt',index_col=False)
# dfx = pd.read_csv('C:\Users\Sarah\Documents\git\Phet-log-analyzer\cck\\raw_data_parsing_check\phet_cck_user_actions+sophistication_WITHPAUSE_more_circuit_info.txt',index_col=False)
df["student"] = df["student"].astype('category')
df["Family"]=df["Family"].str.capitalize()
df["Family_tool"]=df["Family_tool"].str.capitalize()
df["Family_default"]=df["Family_default"].str.capitalize()
df["Family_both"]=df["Family_both"].str.capitalize()

df_scores = pd.read_csv(data_path + 'MATCHING_phet_cck_user_data_anonymized.txt')
df_scores["student"] = df_scores["student"].astype('category')

## Functions we use to get sequences 

In [3]:
converter =  {"Construct" : "C",
"Interface" : "I",
"Pause" : "P",
"Test_basic" : "Tb",
"Test_basic_default" : "Tbd",
"Test_basic_noncontactammeter" : "Tba",
"Test_basic_noncontactammeter_default" : "Tbad",
"Test_basic_noncontactammeter_not" : "Tban",
"Test_basic_not" : "Tbn",
"Test_basic_voltmeter" : "Tbv",
"Test_basic_voltmeter_default" : "Tbvd",
"Test_basic_voltmeter_not" : "Tbvn",
"Test_complex" : "Tc",
"Test_complex_default" : "Tcd",
"Test_complex_noncontactammeter" : "Tca",
"Test_complex_noncontactammeter_default" : "Tcad",
"Test_complex_noncontactammeter_not" : "Tcan",
"Test_complex_not" : "Tcn",
"Test_complex_seriesammeter" : "Tc",
"Test_complex_seriesammeter_default" : "Tcd",
"Test_complex_seriesammeter_not" : "Tcn",
"Test_complex_voltmeter" : "Tcv",
"Test_complex_voltmeter_default" : "Tcvd",
"Test_complex_voltmeter_not" : "Tcvn",
"Test_other" : "To",
"Test_other_default" : "Tod",
"Test_other_noncontactammeter" : "Toa",
"Test_other_noncontactammeter_default" : "Toad",
"Test_other_noncontactammeter_not" : "Toan",
"Test_other_not" : "Ton",
"Test_other_voltmeter_not" : "Tovn",
"Test_other_voltmeter" : "Tov",
"Test_other_voltmeter_default" : "Tovd",
"Test_simple" : "Ts",
"Test_simple_default" : "Tsd",
"Test_simple_noncontactammeter" : "Tsa",
"Test_simple_noncontactammeter_default" : "Tsad",
"Test_simple_noncontactammeter_not" : "Tsan",
"Test_simple_not" : "Tsn",
"Test_simple_voltmeter" : "Tsv",
"Test_simple_voltmeter_default" : "Tsvd",
"Test_simple_voltmeter_not" : "Tsvn",
    }

def get_blocks_withTime_new(df, students, category_column, as_list = True, ignore = [], start = False):
    '''gets blocks of sequences for a list of students
    From the column "Family", "Family_tool", "Family_default" or "Family_both" in the dataframe, each action family is converted to a string in
    the format with at least one capitalized character: 'C', or 'Cccc'.
    To facilitate sequence mining. The sequence is exported as a list:
    ['Ta', 'C','Tb',.....].
    
    Arguments:
        students: list with student ids to generate blocks for
        category_column: the column of the dataframe from which cetagories are taken from
        as_list: by default true. returns sequences as a list of strings instead of a single string
        ignore: list of actions to ignore
        start: if we want a start action to find the first sequence of action of every student   
    
    returns:
     blocks = {student_1_id: ['Ta', 'C','Tb',.....], student_2_id: [...]}    
     time_coords = {student_1_id: [(start_of_action_1, duration), (start_of_action_2, duration),...], student_2_id: [...]}    
    '''
    def convert(action):
        return converter[action]
    
    if start:
        blocks = {student:'S' for student in students}
    else:
        blocks = {student:'' for student in students}

    time_coords = {student:[] for student in students}
    for student in students:
        sequence =  list(df[df['student']==student][category_column])
        time_stamps =  list(df[df['student']==student]['Time Stamp'])
        time_stamps = (time_stamps - min(time_stamps))/1000.  #human readable seconds
        time_coord=[]  #coordinate array for broken bar plot, takes array of (start time, duration)
        p = re.compile(r'([A-Z][a-z]{0,3})\1*')  #this regex finds all action blocks of length 1+
        #print ''.join([convert(action) for action in sequence])
        #print time_stamps
        #use finditer to return a sequence of matches as an iterator
        previous_end = 0
        for match in p.finditer(''.join([convert(action) for action in sequence if convert(action) not in ignore])):
            ind = match.span()  #this gives start and end of matched block
            #for matches of action denoted by more than 1 letter, need to correct the span
            ind = (previous_end, previous_end + (ind[1]-ind[0])/len(set(match.group())))
            #save the end time of one action as the start action of the next
            previous_end = ind[1]
            #print match.group(), ind
            if ind[1] >= len(time_stamps):  #block location offset from real index by 1
                duration = time_stamps[ind[1]-1] - time_stamps[ind[0]]  #time duration of block
                #print time_stamps[ind[1]-1], time_stamps[ind[0]]  #time duration of block
            else:
                duration = time_stamps[ind[1]] - time_stamps[ind[0]]
                #print time_stamps[ind[1]] , time_stamps[ind[0]]
            time_coord.append((time_stamps[ind[0]],duration))
            #print match.group(), match.span(), duration
        #actual regex that converts block of similar actions to just one action
        block = re.sub(r'([A-Z][a-z]{0,3})\1+', r'\1',''.join([convert(action) for action in sequence if convert(action) not in ignore]))
        if as_list:            
            list_block = block[0] + ''.join([' ' + c if c.isupper() else c for c in block[1:]])
            blocks[student] = list_block.split(' ')
            time_coords[student] = time_coord
        else:
            blocks[student] += block
            time_coords[student] = time_coord
    return blocks, time_coords

def get_frequencies(blocks, shortest=3, longest=11):
    '''For each student, given a range of sequence legnths, count how many times students perform each sequence
    Arguments:
        blocks: blocked sequences for each student
        shortest: length of shortest possible mined sequence
        longest: length of longest possible mined sequence
        
    returns:
        frequencies: {student1: Counter{'TPT':3, 'CPT':5...}, ...}
    '''
    frequencies = {student:Counter() for student in blocks.keys()}
    for student,sequence in blocks.iteritems():
        for seq_length in range(shortest, longest+1):  # loops through different possible sequence lengths
            frequencies[student] += Counter(''.join(sequence[i:i+seq_length]) for i in range(len(sequence)-seq_length+1))  # counts string matches for every string of the current length
    return frequencies

def get_bins_per_student(students,time_coords, B):
    ''' For each student, given a certain number of time bins, we find the index of the action that starts each bin.
    
    Arguments:
        students: list with student ids to generate bins for
        B: the number of time bins (typically 5)
        
    returns:
        action_bins = {student_1_id: [ (0,index_action_2nd_bin) , (index_action_2nd_bin,index_action_3rd_bin),... ]}    
    '''
    action_bins = {}
    for student in students:
        action_indices = [None for i in range(1,B)]
        total_time = sum(time_coords[student][-1])
        time_of_bins = [total_time*i/B for i in range(1,B)]
        for i,(t,duration) in enumerate(time_coords[student]):
            for j,b in enumerate(time_of_bins):
                if t <= b:
                    action_indices[j] = i+1
        action_indices.append(len(time_coords[student])) #add last index
        action_indices.insert(0,0)
        #Transform bins in tuples of ranges
        action_bins[student] = [(action_indices[i],action_indices[i+1]) for i in range(0,B)]

    return action_bins

def get_frequencies_by_bin(blocks, students, time_coords, B,shortest=3, longest=11):
    '''
    Given blocks of actions and a range of sequence lengths, we can find the frequency of sequence
    use within each bin.
    
    Arguments:
        blocks: blocked sequences for each student
        action_bins
        shortest: length of shortest possible mined sequence
        longest: length of longest possible mined sequence
        
    returns:
        frequencies = {student: [list of Counters for each bin]}
                    = {student1: [ Counter{'TPT':3, 'CPT':5...}, Counter{},... ],  ...}
    '''
    action_bins = get_bins_per_student(students, time_coords,B)
    frequencies = {student:[Counter() for i in range(B)] for student in blocks.keys()}        
    for student,sequence in blocks.iteritems():
        for seq_length in range(shortest, longest+1):  # loops through different possible sequence lengths
            for j,(start_action,end_action) in enumerate(action_bins[student]): 
                #since we want to find sequence THAT START in bin, we remove the parts of the sequence that fall in previous bins
                portion_of_sequence = sequence[start_action:end_action+seq_length-1]
                frequencies[student][j] += Counter(''.join(portion_of_sequence[i:i+seq_length]) for i in range(end_action-start_action))  # counts string matches for every string of the current length
    return frequencies

def count_use_per_group_per_bin(allfrequencies, frequencies_by_bin, B, attribute, level1, level2):
    '''
    '''
    sequences = allfrequencies.keys()
    counts = {seq : np.zeros((2,B)) for seq in sequences} #initialize empty array for each sequence
    
    student_group_1 = get_students(attribute, level1)
    student_group_2 = get_students(attribute, level2)
    
    for student,f_by_bin in frequencies_by_bin.iteritems():
        for b,counter in enumerate(f_by_bin): 
            for seq in counter: 
                if seq in sequences:
                    if student in student_group_1: 
                        group = 0
                    elif student in student_group_2: 
                        group = 1
                    else:
                        raise Exception("Student not found in groups:{0}".format(stdeunt))
                    counts[seq][group][b] += 1
    return counts


def get_sequence_use_by_timebin(df, students, category_column, B, attribute, level1, level2, shortest_seq_length, longest_seq_length, cut_off):
    '''
    '''
    
    print """Getting sequence use over {3} time bins for {0} students split by {1}. 
    Keeping only sequences used once by at least {2}% of students 
    in each group and overall.""".format(len(students),attribute,int(cut_off*100),B)

    #get all seqs per student per time bin
    blocks, time_coords =  get_blocks_withTime_new(df, students, category_column, start=False, ignore = ['I'])
    frequencies_by_bin = get_frequencies_by_bin(blocks, students, time_coords, B, shortest = shortest_seq_length, longest = longest_seq_length)

    cleaned_frequencies = Counter()
    for attribute,level in [(attribute,level1),(attribute,level2),(None,None)]:
        students = get_students(attribute,level)
        N = int(cut_off*len(students))
        blocks, time_coords =  get_blocks_withTime_new(df, students, category_column, start=False, ignore = ['I'])
        #find all sequences to consider for analysis, given that they have been used by enough students
        frequencies = get_frequencies(blocks, shortest = shortest_seq_length, longest = longest_seq_length)
        counts_frequencies = Counter({f:sum([ 1 if f in freq else 0 for freq in frequencies.values()]) for f in list(sum(frequencies.values(),Counter()))})
        cleaned_frequencies += remove_rare_frequencies(counts_frequencies, N)
    
    counts = count_use_per_group_per_bin(cleaned_frequencies, frequencies_by_bin, B, attribute, level1, level2)
    return counts

### For a particular set of students, we find their sequences of actions (in block format) and find the time coordinate and duration of each action

In [4]:
students = [23112113]
# students = get_students()
B = 4  #number of bins
CUT_OFF_SEQ_USE = 0.20 #we keep only sequences used once by at least 20% of students
N = int(CUT_OFF_SEQ_USE*len(students))
shortest_seq_length = 3
longest_seq_length = 12
blocks, time_coords =  get_blocks_withTime_new(df,students,"Family",start=False,ignore='I')
print blocks, time_coords

{23112113: ['C', 'P', 'C', 'P', 'C', 'P', 'C', 'P', 'C', 'P', 'C', 'Ts', 'C', 'Ts', 'C', 'P', 'C', 'Tc', 'C', 'Ts', 'P', 'C', 'Ts', 'P', 'C', 'Tc', 'P', 'C', 'P', 'C', 'P', 'C', 'Ts', 'P', 'C', 'Ts', 'C', 'P', 'C', 'P', 'C']} {23112113: [(0.0, 86.063000000000002), (86.063000000000002, 0.00099999999999056399), (86.063999999999993, 44.968000000000018), (131.03200000000001, 0.5619999999999834), (131.59399999999999, 76.359000000000009), (207.953, 1.671999999999997), (209.625, 77.577999999999975), (287.20299999999997, 1.438000000000045), (288.64100000000002, 154.875), (443.51600000000002, 2.6409999999999627), (446.15699999999998, 81.593000000000018), (527.75, 9.5470000000000255), (537.29700000000003, 15.312999999999988), (552.61000000000001, 12.717999999999961), (565.32799999999997, 11.735000000000014), (577.06299999999999, 1.3279999999999745), (578.39099999999996, 76.969000000000051), (655.36000000000001, 10.217999999999961), (665.57799999999997, 7.6570000000000391), (673.23500000000001, 1

### Now we can get the frequency of use of sequences for each student to then:
* Count number of time sequences occurs accross all students
* Count unique number of students that conducted that sequence

In [5]:
frequencies = get_frequencies(blocks, shortest=shortest_seq_length, longest=longest_seq_length)
#Count number of time sequences occurs accross all students
sum_frequencies = sum(frequencies.values(), Counter())
#Count unique number of students that conducted that sequence
counts_frequencies = Counter({f:sum([ 1 if f in freq else 0 for freq in frequencies.values()]) for f in list(sum_frequencies)})

### Now we can select certain kinds of sequences such as:
* keep all sequences with a certain action
* remove rarely used sequences
then the most common sequences can be printed

In [6]:
N = int(CUT_OFF_SEQ_USE*len(blocks.keys()))
counts_frequencies1 = remove_rare_frequencies(counts_frequencies,N)
counts_frequencies2 = keep_frequencies_with(counts_frequencies,keep='P')
counts_frequencies2.most_common(10)
counts_frequencies1

Counter({'CPC': 1,
         'CPCP': 1,
         'CPCPC': 1,
         'CPCPCP': 1,
         'CPCPCPC': 1,
         'CPCPCPCP': 1,
         'CPCPCPCPC': 1,
         'CPCPCPCPCP': 1,
         'CPCPCPCPCPC': 1,
         'CPCPCPCPCPCTs': 1,
         'CPCPCPCPCTs': 1,
         'CPCPCPCPCTsC': 1,
         'CPCPCPCPCTsCTs': 1,
         'CPCPCPCTs': 1,
         'CPCPCPCTsC': 1,
         'CPCPCPCTsCTs': 1,
         'CPCPCPCTsCTsC': 1,
         'CPCPCPCTsCTsCP': 1,
         'CPCPCTs': 1,
         'CPCPCTsC': 1,
         'CPCPCTsCTs': 1,
         'CPCPCTsCTsC': 1,
         'CPCPCTsCTsCP': 1,
         'CPCPCTsCTsCPC': 1,
         'CPCPCTsCTsCPCTc': 1,
         'CPCPCTsP': 1,
         'CPCPCTsPC': 1,
         'CPCPCTsPCTs': 1,
         'CPCPCTsPCTsC': 1,
         'CPCPCTsPCTsCP': 1,
         'CPCPCTsPCTsCPC': 1,
         'CPCTc': 1,
         'CPCTcC': 1,
         'CPCTcCTs': 1,
         'CPCTcCTsP': 1,
         'CPCTcCTsPC': 1,
         'CPCTcCTsPCTs': 1,
         'CPCTcCTsPCTsP': 1,
         'CPCTc

#### For each student, given a certain number of time bins, we find their sequences of actions within bins.
The three sections below were used to debug the frequencies_by_bin function such that sequences must START in a time bin but not end there.

In [7]:
frequencies_by_bin = get_frequencies_by_bin(blocks, students, time_coords, B, shortest = shortest_seq_length, longest = 5)
print ''.join(blocks[students[0]])
f1 = sum(frequencies_by_bin[students[0]],Counter())
f2 = frequencies[students[0]]
print f1,'\n'
print get_bins_per_student(students,time_coords, B)[students[0]]
# print f2
# for counts in f1:
#     for c in counts:
#         if c not in f2:
#             print c
"PCTsCTs" in f1
print frequencies_by_bin[students[0]]

CPCPCPCPCPCTsCTsCPCTcCTsPCTsPCTcPCPCPCTsPCTsCPCPC
Counter({'CPC': 12, 'PCPC': 8, 'PCP': 7, 'CPCPC': 6, 'CPCP': 6, 'PCPCP': 4, 'PCTs': 4, 'PC': 3, 'CTsC': 3, 'TsPC': 3, 'CTsP': 3, 'C': 3, 'CTsPC': 3, 'PCTc': 2, 'PCTsPC': 2, 'CPCTs': 2, 'CTsCP': 2, 'PCPCTs': 2, 'TsPCTs': 2, 'CTsCPC': 2, 'TsCPC': 2, 'TsCP': 2, 'CTsPCTs': 2, 'PCTsP': 2, 'PCTsC': 2, 'CPCTc': 1, 'TsPCTsC': 1, 'PCTsCTs': 1, 'CPCTcC': 1, 'TcCTsPC': 1, 'TsPCTsP': 1, 'TcCTs': 1, 'CTcPC': 1, 'TsPCTcP': 1, 'CTsCTsC': 1, 'CTcCTs': 1, 'CTcCTsP': 1, 'TcPCPC': 1, 'TsPCTc': 1, 'TsCTsC': 1, 'CPCTsC': 1, 'CTcP': 1, 'PCTcC': 1, 'PCTcPC': 1, 'CPCTsP': 1, 'PCTcP': 1, 'CTcC': 1, 'TcCTsP': 1, 'PCTcCTs': 1, 'TcPC': 1, 'CTcPCP': 1, 'PCTsCP': 1, 'TsCTs': 1, 'TsCPCP': 1, 'CTsPCTc': 1, 'TcPCP': 1, 'TsCPCTc': 1, 'CTsCTs': 1, 'TsCTsCP': 1}) 

[(0, 9), (9, 17), (17, 34), (34, 41)]
[Counter({'CPC': 5, 'PCP': 4, 'CPCP': 4, 'PCPC': 4, 'CPCPC': 4, 'PCPCP': 3, 'CPCTsC': 1, 'PCPCTs': 1, 'CPCTs': 1}), Counter({'CTsC': 2, 'PCTc': 1, 'CPCTc': 1, 'CPC': 1, 'PC

In [8]:
s = "CPCTsATsCTcCTv"
len(s)

14

In [11]:
sequence = ['P','C','P','C','Ts','A','Ts','C','Tc','C','Tv']
print ''.join(sequence)
action_bins = [(0,4),(4,7),(7,11)]
B = 3
shortest = 2
longest = 4

frequencies1 = [Counter() for i in range(B)]        
frequencies2 = [Counter() for i in range(B)]        
for seq_length in range(shortest, longest+1):  # loops through different possible sequence lengths
    for j,(start_action,end_action) in enumerate(action_bins): 
        #since we want to find sequence THAT START in bin, we remove the parts of the sequence that fall in previous bins
        portion_of_sequence = sequence[start_action:end_action+seq_length-1]
        print start_action,end_action
        where_must_start = sequence[start_action:end_action]
        print j,portion_of_sequence, where_must_start
        print range(len(where_must_start))
        print Counter(''.join(portion_of_sequence[i:i+seq_length]) for i in range(end_action-start_action))  # counts string matches for every string of the current length

#         frequencies1[j] 
# frequencies2[j] 
# print frequencies1
# print frequencies2

PCPCTsATsCTcCTv
0 4
0 ['P', 'C', 'P', 'C', 'Ts'] ['P', 'C', 'P', 'C']
[0, 1, 2, 3]
Counter({'PC': 2, 'CP': 1, 'CTs': 1})
4 7
1 ['Ts', 'A', 'Ts', 'C'] ['Ts', 'A', 'Ts']
[0, 1, 2]
Counter({'TsC': 1, 'ATs': 1, 'TsA': 1})
7 11
2 ['C', 'Tc', 'C', 'Tv'] ['C', 'Tc', 'C', 'Tv']
[0, 1, 2, 3]
Counter({'Tv': 1, 'CTv': 1, 'TcC': 1, 'CTc': 1})
0 4
0 ['P', 'C', 'P', 'C', 'Ts', 'A'] ['P', 'C', 'P', 'C']
[0, 1, 2, 3]
Counter({'PCTs': 1, 'CTsA': 1, 'PCP': 1, 'CPC': 1})
4 7
1 ['Ts', 'A', 'Ts', 'C', 'Tc'] ['Ts', 'A', 'Ts']
[0, 1, 2]
Counter({'TsCTc': 1, 'ATsC': 1, 'TsATs': 1})
7 11
2 ['C', 'Tc', 'C', 'Tv'] ['C', 'Tc', 'C', 'Tv']
[0, 1, 2, 3]
Counter({'Tv': 1, 'TcCTv': 1, 'CTv': 1, 'CTcC': 1})
0 4
0 ['P', 'C', 'P', 'C', 'Ts', 'A', 'Ts'] ['P', 'C', 'P', 'C']
[0, 1, 2, 3]
Counter({'PCTsA': 1, 'CTsATs': 1, 'CPCTs': 1, 'PCPC': 1})
4 7
1 ['Ts', 'A', 'Ts', 'C', 'Tc', 'C'] ['Ts', 'A', 'Ts']
[0, 1, 2]
Counter({'ATsCTc': 1, 'TsATsC': 1, 'TsCTcC': 1})
7 11
2 ['C', 'Tc', 'C', 'Tv'] ['C', 'Tc', 'C', 'Tv']
[0, 1, 2, 3